# 含并行连结的网络（GoogLeNet）

* 解决了什么样大小的卷积核最合适的问题
* 有时使用不同大小的卷积核组合是有利的

## Inception块

* 在GoogLeNet中，基本的卷积块被称为*Inception块*（Inception block）
* We need to go deeper
* 通常调整的超参数是每层输出通道数
* 阅读：含并行连结的网络\_GoogLeNet.pdf

<center><img src="../img/inception.svg" width="50%"></center>
<center>Inception块的架构</center>

## Inception块

* 前三条路径使用窗口大小为$1\times 1$、$3\times 3$和$5\times 5$的卷积层，从不同空间大小中提取信息
* 中间的两条路径在输入上执行$1\times 1$卷积，以减少通道数，从而降低模型的复杂性

In [3]:
import torch
from d2l import torch as d2l
from torch import nn
from torch.nn import functional as F


class Inception(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大池化层后接1x1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat((p1, p2, p3, p4), dim=1)

## Inception块

* To gain some intuition for why this network works so well, consider the combination of the filters. 
* They explore the image in a variety of filter sizes. 
* This means that details at different extents can be recognized efficiently by filters of different sizes. 
* At the same time, we can allocate different amounts of parameters for different filters.

## GoogLeNet模型

* GoogLeNet一共使用9个Inception块和全局平均池化层的堆叠来生成其估计值

<center><img src="../img/inception-full.svg" width="15%"></center>
<center>GoogLeNet架构</center>

## GoogLeNet模型

* Inception块之间的最大池化层可降低维度
* 第一个模块类似于AlexNet和LeNet，Inception块的组合从VGG继承，全局平均池化层避免了在最后使用全连接层
* 阅读：含并行连结的网络\_GoogLeNet.pdf
* 视频学习：[段3](https://www.bilibili.com/video/BV1b5411g7Xo/)

## 小结

* Inception块的通道数分配之比是在ImageNet数据集上通过大量的实验得来的
* GoogLeNet和它的后继者们一度是ImageNet上最有效的模型之一：它以较低的计算复杂度提供了类似的测试精度